In [1]:
import numpy as np
import pandas as pd

In [2]:
# load data

dataX=pd.read_csv("./data/trainX.csv")
dataA=pd.read_csv("./data/trainA.csv")
dataY=pd.read_csv("./data/trainY.csv")

# remove index

dataX=dataX.values[:,1:]
dataY=dataY.values[:,1:-1]
dataA=dataA.values[:,1:]

In [3]:
# split cure or not cure
치료한환자데이터=pd.read_csv("./data/2번 열을 제거한 치료한 환자 데이터.csv")
치료하지않은환자데이터=pd.read_csv("./data/8번 열을 제거한 치료하지 않은 환자 데이터.csv")

In [4]:
data_CY=치료한환자데이터['time']
dataNCY=치료하지않은환자데이터['time']
data_CX = 치료한환자데이터.loc[:,['X0','X1','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16']]
dataNCX = 치료하지않은환자데이터.loc[:,['X0','X1','X2','X3','X4','X5','X6','X7','X9','X10','X11','X12','X13','X14','X15','X16']]

In [5]:
from keras import models
from keras import layers

Using TensorFlow backend.
c:\users\kosta\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\kosta\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\kosta\anaconda3\envs\generative\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\kosta\anaconda3\envs\ge

In [6]:
import tensorflow as tf

In [25]:
def build_dnn_relu_adam():
  # 2. 모델의 구조를 BatchNormalization layer를 사용하여 만든다.
  X = tf.keras.layers.Input(shape=[16])
  
  H = tf.keras.layers.Dense(512)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 1
  
  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 2
  
  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 3
    
  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 4
  
  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 5

  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 6

  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('relu')(H) # Hidden layer 7

  Y = tf.keras.layers.Dense(1)(H)
  model = tf.keras.models.Model(X, Y)
  model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()])
  return model

def build_model_NC_100_stepper():
  # 2. 모델의 구조를 BatchNormalization layer를 사용하여 만든다.
  X = tf.keras.layers.Input(shape=[16])
  
  H = tf.keras.layers.Dense(2048)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)
  
  H = tf.keras.layers.Dense(1024)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)
  
  H = tf.keras.layers.Dense(512)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)

  H = tf.keras.layers.Dense(256)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)

  H = tf.keras.layers.Dense(128)(H)
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('swish')(H)
  
  Y = tf.keras.layers.Dense(1)(H)
  model = tf.keras.models.Model(X, Y)
  model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()])
  return model

In [21]:
# 치료 데이터
num_epochs = 20000
all_mae_histories = []
k = 4
num_val_samples = len(data_CX) // k
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = data_CX[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = data_CY[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [data_CX[:i * num_val_samples],
         data_CX[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [data_CY[:i * num_val_samples],
         data_CY[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_dnn_relu_adam()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['mean_absolute_error']
    all_mae_histories.append(mae_history)

처리중인 폴드 # 0


KeyError: 'mean_absolute_error'

In [ ]:
# 치료하지 않은 데이터
num_epochs = 20000
all_mae_histories_non_cured = []
k = 4
num_val_samples = len(dataNCX) // k
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = dataNCX[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = dataNCY[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [dataNCX[:i * num_val_samples],
         dataNCX[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [dataNCY[:i * num_val_samples],
         dataNCY[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_dnn_swish_adam()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history_non_cured = history.history['mean_absolute_error']
    all_mae_histories_non_cured.append(mae_history)

In [26]:
# 3.데이터로 모델을 학습(FIT)합니다.
model = build_dnn_relu_adam() # 최저 loss 값 : 0.1506
model.fit(data_CX, data_CY, epochs=10000, verbose=1)

Epoch 1/10000
264/264 [==============================] - 2s 7ms/sample - loss: 86.9867 - accuracy: 0.0000e+00
Epoch 2/10000
264/264 [==============================] - 0s 910us/sample - loss: 48.9967 - accuracy: 0.0000e+00
Epoch 3/10000
264/264 [==============================] - 0s 971us/sample - loss: 20.1581 - accuracy: 0.0000e+00
Epoch 4/10000
264/264 [==============================] - 0s 737us/sample - loss: 19.2519 - accuracy: 0.0000e+00
Epoch 5/10000
264/264 [==============================] - 0s 740us/sample - loss: 19.7436 - accuracy: 0.0000e+00
Epoch 6/10000
264/264 [==============================] - 0s 714us/sample - loss: 35.9295 - accuracy: 0.0000e+00
Epoch 7/10000
264/264 [==============================] - 0s 759us/sample - loss: 18.2813 - accuracy: 0.0000e+00
Epoch 8/10000
264/264 [==============================] - 0s 718us/sample - loss: 30.0105 - accuracy: 0.0000e+00
Epoch 9/10000
264/264 [==============================] - 0s 733us/sample - loss: 17.3280 - accuracy: 0.000

264/264 [==============================] - 0s 888us/sample - loss: 3.8426 - accuracy: 0.0000e+00
Epoch 75/10000
264/264 [==============================] - 0s 748us/sample - loss: 7.0942 - accuracy: 0.0000e+00
Epoch 76/10000
264/264 [==============================] - 0s 774us/sample - loss: 8.7243 - accuracy: 0.0000e+00
Epoch 77/10000
264/264 [==============================] - 0s 842us/sample - loss: 9.9834 - accuracy: 0.0000e+00
Epoch 78/10000
264/264 [==============================] - 0s 748us/sample - loss: 5.0440 - accuracy: 0.0000e+00
Epoch 79/10000
264/264 [==============================] - 0s 748us/sample - loss: 4.6940 - accuracy: 0.0000e+00
Epoch 80/10000
264/264 [==============================] - 0s 895us/sample - loss: 5.3486 - accuracy: 0.0000e+00
Epoch 81/10000
264/264 [==============================] - 0s 918us/sample - loss: 5.7972 - accuracy: 0.0000e+00
Epoch 82/10000
264/264 [==============================] - 0s 827us/sample - loss: 4.9294 - accuracy: 0.0000e+00
Epoch 8

264/264 [==============================] - 0s 797us/sample - loss: 13.4944 - accuracy: 0.0000e+00
Epoch 147/10000
264/264 [==============================] - 0s 801us/sample - loss: 7.9374 - accuracy: 0.0000e+00
Epoch 148/10000
264/264 [==============================] - 0s 824us/sample - loss: 7.9579 - accuracy: 0.0000e+00
Epoch 149/10000
264/264 [==============================] - 0s 774us/sample - loss: 6.9900 - accuracy: 0.0000e+00
Epoch 150/10000
264/264 [==============================] - 0s 737us/sample - loss: 10.4826 - accuracy: 0.0000e+00
Epoch 151/10000
264/264 [==============================] - 0s 740us/sample - loss: 6.7974 - accuracy: 0.0000e+00
Epoch 152/10000
264/264 [==============================] - 0s 737us/sample - loss: 6.3135 - accuracy: 0.0000e+00
Epoch 153/10000
264/264 [==============================] - 0s 737us/sample - loss: 6.2377 - accuracy: 0.0000e+00
Epoch 154/10000
264/264 [==============================] - 0s 756us/sample - loss: 7.2954 - accuracy: 0.0000e+

264/264 [==============================] - 0s 903us/sample - loss: 14.3805 - accuracy: 0.0000e+00
Epoch 219/10000
264/264 [==============================] - 0s 846us/sample - loss: 14.0844 - accuracy: 0.0000e+00
Epoch 220/10000
264/264 [==============================] - 0s 873us/sample - loss: 8.6466 - accuracy: 0.0000e+00
Epoch 221/10000
264/264 [==============================] - 0s 873us/sample - loss: 6.6530 - accuracy: 0.0000e+00
Epoch 222/10000
264/264 [==============================] - 0s 854us/sample - loss: 8.5650 - accuracy: 0.0000e+00
Epoch 223/10000
264/264 [==============================] - 0s 831us/sample - loss: 4.5439 - accuracy: 0.0000e+00
Epoch 224/10000
264/264 [==============================] - 0s 808us/sample - loss: 4.8678 - accuracy: 0.0000e+00
Epoch 225/10000
264/264 [==============================] - 0s 778us/sample - loss: 5.3489 - accuracy: 0.0000e+00
Epoch 226/10000
264/264 [==============================] - 0s 756us/sample - loss: 3.8581 - accuracy: 0.0000e+

264/264 [==============================] - 0s 975us/sample - loss: 1.7355 - accuracy: 0.0000e+00
Epoch 291/10000
264/264 [==============================] - 0s 967us/sample - loss: 2.4066 - accuracy: 0.0000e+00
Epoch 292/10000
264/264 [==============================] - 0s 839us/sample - loss: 4.9499 - accuracy: 0.0000e+00
Epoch 293/10000
264/264 [==============================] - 0s 899us/sample - loss: 4.0617 - accuracy: 0.0000e+00
Epoch 294/10000
264/264 [==============================] - 0s 1ms/sample - loss: 2.3002 - accuracy: 0.0000e+00
Epoch 295/10000
264/264 [==============================] - 0s 948us/sample - loss: 3.4873 - accuracy: 0.0000e+00
Epoch 296/10000
264/264 [==============================] - 0s 1ms/sample - loss: 2.1629 - accuracy: 0.0000e+00
Epoch 297/10000
264/264 [==============================] - 0s 918us/sample - loss: 2.3327 - accuracy: 0.0000e+00
Epoch 298/10000
264/264 [==============================] - 0s 816us/sample - loss: 3.2943 - accuracy: 0.0000e+00
Epo

...

264/264 [==============================] - 0s 695us/sample - loss: 0.1328 - accuracy: 0.0000e+00
Epoch 9675/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1208 - accuracy: 0.0000e+00
Epoch 9676/10000
264/264 [==============================] - 0s 706us/sample - loss: 0.1045 - accuracy: 0.0000e+00
Epoch 9677/10000
264/264 [==============================] - 0s 706us/sample - loss: 0.1425 - accuracy: 0.0000e+00
Epoch 9678/10000
264/264 [==============================] - 0s 706us/sample - loss: 0.0994 - accuracy: 0.0000e+00
Epoch 9679/10000
264/264 [==============================] - 0s 771us/sample - loss: 0.0820 - accuracy: 0.0000e+00
Epoch 9680/10000
264/264 [==============================] - 0s 748us/sample - loss: 0.1362 - accuracy: 0.0000e+00
Epoch 9681/10000
264/264 [==============================] - 0s 797us/sample - loss: 0.1012 - accuracy: 0.0000e+00
Epoch 9682/10000
264/264 [==============================] - 0s 808us/sample - loss: 0.1003 - accuracy: 0.

264/264 [==============================] - 0s 718us/sample - loss: 0.1338 - accuracy: 0.0000e+00
Epoch 9746/10000
264/264 [==============================] - 0s 782us/sample - loss: 0.3156 - accuracy: 0.0000e+00
Epoch 9747/10000
264/264 [==============================] - 0s 793us/sample - loss: 0.2916 - accuracy: 0.0000e+00
Epoch 9748/10000
264/264 [==============================] - 0s 756us/sample - loss: 0.2516 - accuracy: 0.0000e+00
Epoch 9749/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.3417 - accuracy: 0.0000e+00
Epoch 9750/10000
264/264 [==============================] - 0s 687us/sample - loss: 0.1606 - accuracy: 0.0000e+00
Epoch 9751/10000
264/264 [==============================] - 0s 688us/sample - loss: 0.1216 - accuracy: 0.0000e+00
Epoch 9752/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1883 - accuracy: 0.0000e+00
Epoch 9753/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1961 - accuracy: 0.

264/264 [==============================] - 0s 725us/sample - loss: 0.5125 - accuracy: 0.0000e+00
Epoch 9817/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.3590 - accuracy: 0.0000e+00
Epoch 9818/10000
264/264 [==============================] - 0s 703us/sample - loss: 0.3554 - accuracy: 0.0000e+00
Epoch 9819/10000
264/264 [==============================] - 0s 733us/sample - loss: 0.1940 - accuracy: 0.0000e+00
Epoch 9820/10000
264/264 [==============================] - 0s 797us/sample - loss: 0.2567 - accuracy: 0.0000e+00
Epoch 9821/10000
264/264 [==============================] - 0s 752us/sample - loss: 0.1753 - accuracy: 0.0000e+00
Epoch 9822/10000
264/264 [==============================] - 0s 846us/sample - loss: 0.1324 - accuracy: 0.0000e+00
Epoch 9823/10000
264/264 [==============================] - 0s 842us/sample - loss: 0.2119 - accuracy: 0.0000e+00
Epoch 9824/10000
264/264 [==============================] - 0s 790us/sample - loss: 0.2618 - accuracy: 0.

264/264 [==============================] - 0s 688us/sample - loss: 0.2127 - accuracy: 0.0000e+00
Epoch 9888/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1400 - accuracy: 0.0000e+00
Epoch 9889/10000
264/264 [==============================] - 0s 684us/sample - loss: 0.1168 - accuracy: 0.0000e+00
Epoch 9890/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.3059 - accuracy: 0.0000e+00
Epoch 9891/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1349 - accuracy: 0.0000e+00
Epoch 9892/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.1692 - accuracy: 0.0000e+00
Epoch 9893/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.1474 - accuracy: 0.0000e+00
Epoch 9894/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1408 - accuracy: 0.0000e+00
Epoch 9895/10000
264/264 [==============================] - 0s 683us/sample - loss: 0.1097 - accuracy: 0.

264/264 [==============================] - 0s 710us/sample - loss: 0.0663 - accuracy: 0.0000e+00
Epoch 9959/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.1029 - accuracy: 0.0000e+00
Epoch 9960/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.0771 - accuracy: 0.0000e+00
Epoch 9961/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.0697 - accuracy: 0.0000e+00
Epoch 9962/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.1223 - accuracy: 0.0000e+00
Epoch 9963/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.0790 - accuracy: 0.0000e+00
Epoch 9964/10000
264/264 [==============================] - 0s 686us/sample - loss: 0.0981 - accuracy: 0.0000e+00
Epoch 9965/10000
264/264 [==============================] - 0s 688us/sample - loss: 0.1042 - accuracy: 0.0000e+00 - loss: 0.0937 - accuracy: 0.0000e+
Epoch 9966/10000
264/264 [==============================] - 0s 695us/

In [27]:
# 3.치료하지 않았고 생존시간이 100 이하인 환자 데이터로 모델을 학습(FIT)합니다.
model_NC = build_dnn_relu_adam() # 최저 loss 값 : 0.0607
model.fit(data_CX, data_CY, epochs=10000, verbose=1)

Epoch 1/10000
264/264 [==============================] - 0s 778us/sample - loss: 0.1820 - accuracy: 0.0000e+00
Epoch 2/10000
264/264 [==============================] - 0s 786us/sample - loss: 0.1213 - accuracy: 0.0000e+00
Epoch 3/10000
264/264 [==============================] - 0s 793us/sample - loss: 0.1716 - accuracy: 0.0000e+00
Epoch 4/10000
264/264 [==============================] - 0s 722us/sample - loss: 0.1127 - accuracy: 0.0000e+00
Epoch 5/10000
264/264 [==============================] - 0s 710us/sample - loss: 0.1255 - accuracy: 0.0000e+00
Epoch 6/10000
264/264 [==============================] - 0s 710us/sample - loss: 0.1258 - accuracy: 0.0000e+00
Epoch 7/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.0836 - accuracy: 0.0000e+00
Epoch 8/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.0999 - accuracy: 0.0000e+00
Epoch 9/10000
264/264 [==============================] - 0s 691us/sample - loss: 1.8301 - accuracy: 0.0000e+00
E

264/264 [==============================] - 0s 756us/sample - loss: 0.1149 - accuracy: 0.0000e+00
Epoch 74/10000
264/264 [==============================] - 0s 801us/sample - loss: 0.0969 - accuracy: 0.0000e+00
Epoch 75/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.0933 - accuracy: 0.0000e+00
Epoch 76/10000
264/264 [==============================] - 0s 752us/sample - loss: 0.0961 - accuracy: 0.0000e+00
Epoch 77/10000
264/264 [==============================] - 0s 797us/sample - loss: 0.0843 - accuracy: 0.0000e+00
Epoch 78/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.0883 - accuracy: 0.0000e+00
Epoch 79/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.0873 - accuracy: 0.0000e+00 - loss: 0.0762 - accuracy: 0.0000e+
Epoch 80/10000
264/264 [==============================] - 0s 705us/sample - loss: 0.0655 - accuracy: 0.0000e+00
Epoch 81/10000
264/264 [==============================] - 0s 688us/sample - loss: 0

264/264 [==============================] - 0s 694us/sample - loss: 0.1428 - accuracy: 0.0000e+00
Epoch 145/10000
264/264 [==============================] - 0s 686us/sample - loss: 0.1822 - accuracy: 0.0000e+00
Epoch 146/10000
264/264 [==============================] - 0s 688us/sample - loss: 0.1165 - accuracy: 0.0000e+00
Epoch 147/10000
264/264 [==============================] - 0s 688us/sample - loss: 0.1103 - accuracy: 0.0000e+00
Epoch 148/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.1422 - accuracy: 0.0000e+00
Epoch 149/10000
264/264 [==============================] - 0s 692us/sample - loss: 0.1416 - accuracy: 0.0000e+00
Epoch 150/10000
264/264 [==============================] - 0s 684us/sample - loss: 0.1093 - accuracy: 0.0000e+00
Epoch 151/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.1145 - accuracy: 0.0000e+00
Epoch 152/10000
264/264 [==============================] - 0s 733us/sample - loss: 0.1123 - accuracy: 0.0000e+00

264/264 [==============================] - 0s 722us/sample - loss: 0.1339 - accuracy: 0.0000e+00
Epoch 217/10000
264/264 [==============================] - 0s 688us/sample - loss: 0.4938 - accuracy: 0.0000e+00
Epoch 218/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.3008 - accuracy: 0.0000e+00
Epoch 219/10000
264/264 [==============================] - 0s 698us/sample - loss: 0.1819 - accuracy: 0.0000e+00
Epoch 220/10000
264/264 [==============================] - ETA: 0s - loss: 0.1764 - accuracy: 0.0000e+00 ETA: 0s - loss: 0.2010 - accuracy: 0.0000 - 0s 684us/sample - loss: 0.2068 - accuracy: 0.0000e+00
Epoch 221/10000
264/264 [==============================] - 0s 744us/sample - loss: 0.1962 - accuracy: 0.0000e+00
Epoch 222/10000
264/264 [==============================] - 0s 790us/sample - loss: 0.1829 - accuracy: 0.0000e+00
Epoch 223/10000
264/264 [==============================] - 0s 774us/sample - loss: 0.2314 - accuracy: 0.0000e+00
Epoch 224/10000
264/26

264/264 [==============================] - 0s 688us/sample - loss: 0.5972 - accuracy: 0.0000e+00
Epoch 288/10000
264/264 [==============================] - 0s 706us/sample - loss: 1.0260 - accuracy: 0.0000e+00
Epoch 289/10000
264/264 [==============================] - 0s 684us/sample - loss: 0.4326 - accuracy: 0.0000e+00
Epoch 290/10000
264/264 [==============================] - 0s 685us/sample - loss: 0.3066 - accuracy: 0.0000e+00
Epoch 291/10000
264/264 [==============================] - 0s 688us/sample - loss: 0.2348 - accuracy: 0.0000e+00
Epoch 292/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.2213 - accuracy: 0.0000e+00
Epoch 293/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.3037 - accuracy: 0.0000e+00
Epoch 294/10000
264/264 [==============================] - 0s 695us/sample - loss: 0.3236 - accuracy: 0.0000e+00
Epoch 295/10000
264/264 [==============================] - 0s 691us/sample - loss: 0.1999 - accuracy: 0.0000e+00

...

264/264 [==============================] - 0s 759us/sample - loss: 0.1628 - accuracy: 0.0000e+00
Epoch 4269/10000
264/264 [==============================] - 0s 714us/sample - loss: 0.1965 - accuracy: 0.0000e+00
Epoch 4270/10000
264/264 [==============================] - 0s 714us/sample - loss: 0.0960 - accuracy: 0.0000e+00
Epoch 4271/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.2133 - accuracy: 0.0000e+00
Epoch 4272/10000
264/264 [==============================] - 0s 710us/sample - loss: 1.0298 - accuracy: 0.0000e+00
Epoch 4273/10000
264/264 [==============================] - 0s 716us/sample - loss: 0.4159 - accuracy: 0.0000e+00
Epoch 4274/10000
264/264 [==============================] - 0s 801us/sample - loss: 0.5355 - accuracy: 0.0000e+00
Epoch 4275/10000
264/264 [==============================] - 0s 780us/sample - loss: 0.3585 - accuracy: 0.0000e+00
Epoch 4276/10000
264/264 [==============================] - 0s 816us/sample - loss: 0.4503 - accuracy: 0.

264/264 [==============================] - 0s 733us/sample - loss: 0.0954 - accuracy: 0.0000e+00
Epoch 4341/10000
264/264 [==============================] - 0s 801us/sample - loss: 0.0787 - accuracy: 0.0000e+00
Epoch 4342/10000
264/264 [==============================] - 0s 752us/sample - loss: 0.0631 - accuracy: 0.0000e+00
Epoch 4343/10000
264/264 [==============================] - 0s 703us/sample - loss: 0.1063 - accuracy: 0.0000e+00
Epoch 4344/10000
264/264 [==============================] - 0s 697us/sample - loss: 0.0799 - accuracy: 0.0000e+00
Epoch 4345/10000
264/264 [==============================] - 0s 793us/sample - loss: 0.2888 - accuracy: 0.0000e+00
Epoch 4346/10000
264/264 [==============================] - 0s 782us/sample - loss: 0.2492 - accuracy: 0.0000e+00
Epoch 4347/10000
264/264 [==============================] - 0s 710us/sample - loss: 0.4004 - accuracy: 0.0000e+00
Epoch 4348/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.1619 - accuracy: 0.

264/264 [==============================] - 0s 714us/sample - loss: 0.0918 - accuracy: 0.0000e+00
Epoch 4413/10000
264/264 [==============================] - 0s 699us/sample - loss: 0.4094 - accuracy: 0.0000e+00
Epoch 4414/10000
264/264 [==============================] - 0s 740us/sample - loss: 0.1607 - accuracy: 0.0000e+00
Epoch 4415/10000
264/264 [==============================] - 0s 706us/sample - loss: 0.5200 - accuracy: 0.0000e+00
Epoch 4416/10000
264/264 [==============================] - 0s 752us/sample - loss: 0.4639 - accuracy: 0.0000e+00
Epoch 4417/10000
264/264 [==============================] - 0s 778us/sample - loss: 0.3344 - accuracy: 0.0000e+00
Epoch 4418/10000
264/264 [==============================] - 0s 714us/sample - loss: 0.1797 - accuracy: 0.0000e+00
Epoch 4419/10000
264/264 [==============================] - 0s 718us/sample - loss: 0.1816 - accuracy: 0.0000e+00
Epoch 4420/10000
264/264 [==============================] - 0s 854us/sample - loss: 0.1389 - accuracy: 0.

264/264 [==============================] - 0s 805us/sample - loss: 0.1615 - accuracy: 0.0000e+00
Epoch 4485/10000
264/264 [==============================] - 0s 748us/sample - loss: 0.1696 - accuracy: 0.0000e+00
Epoch 4486/10000
264/264 [==============================] - 0s 729us/sample - loss: 0.1479 - accuracy: 0.0000e+00
Epoch 4487/10000
264/264 [==============================] - 0s 740us/sample - loss: 0.1773 - accuracy: 0.0000e+00
Epoch 4488/10000
264/264 [==============================] - 0s 733us/sample - loss: 0.1210 - accuracy: 0.0000e+00
Epoch 4489/10000
264/264 [==============================] - 0s 767us/sample - loss: 0.2182 - accuracy: 0.0000e+00
Epoch 4490/10000
264/264 [==============================] - 0s 729us/sample - loss: 0.1305 - accuracy: 0.0000e+00
Epoch 4491/10000
264/264 [==============================] - 0s 740us/sample - loss: 0.1240 - accuracy: 0.0000e+00
Epoch 4492/10000
264/264 [==============================] - 0s 725us/sample - loss: 0.1168 - accuracy: 0.

KeyboardInterrupt: 

In [22]:
치료_환자_파일경로 = './data/2번 열을 제거한 치료한 환자 테스트 데이터.csv'
비치료_환자_파일경로 = './data/8번 열을 제거한 치료하지 않은 환자 테스트 데이터.csv'
치료_환자_데이터 = pd.read_csv(치료_환자_파일경로)
비치료_환자_데이터 = pd.read_csv(비치료_환자_파일경로)

In [23]:
# 치료할 경우 생존 시간을 구한다.
cured = model.predict(치료_환자_데이터)
# 치료하지 않을 경우 생존 시간을 구한다.
non_cured = model_NC.predict(비치료_환자_데이터)

# 치료한 경우 생존 시간 > 치료하지 않은 경우 생존 시간 : 1, 
# 치료한 경우 생존 시간 > 치료하지 않은 경우 생존 시간 : 0
result = np.where(cured > non_cured, 1, 0)
print(result.shape)

(286, 1)


In [24]:
import csv 

# 이차원 리스트 
with open('./result_20200903_3.csv','w', newline='') as f: 
  makewrite = csv.writer(f) 
  
  for value in result: 
    makewrite.writerow(value)
